# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="Tennis_Windows_x86_64/Tennis.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [5]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

Score (max over agents) from episode 1: 0.0
Score (max over agents) from episode 2: 0.0
Score (max over agents) from episode 3: 0.0
Score (max over agents) from episode 4: 0.0
Score (max over agents) from episode 5: 0.0


When finished, you can close the environment.

In [6]:
#env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [7]:
# Start from DDPG CODE
# First, library and agent
import random
import torch

import sys
import os

from collections import deque
import time
#sys.path.append(os.path.abspath('python/'))
from ma_agent import ma_agent

# start an ma-agent of ddpg for multiple-agents
#----------need change---------------#
ma_ddpg = ma_agent(num_agents=num_agents, state_size=state_size, action_size=action_size)
dir_chkpoints='dir_chkpoints'
# Function for the model record for actor and critic networks for both agents
def save_weights(dir):
    for i in range(num_agents):
        torch.save(ma_ddpg.agents[i].actor_local.state_dict(),                     
              os.path.join(dir, 'checkpoint_actor_{}.pth'.format(i)) )
        torch.save(ma_ddpg.agents[i].critic_local.state_dict(), 
              os.path.join(dir, 'checkpoint_critic_{}.pth'.format(i)) )


In [11]:
# ma training is a bit different from that of ddpg
def ma_ddpg_train(n_episodes=10000,save_every=50 ,print_every=20):
    solved = False
    scores_total = []
    scores_deque = deque(maxlen=100)
    scores_global = []
    best_score = 0.0
    time_start = time.time()    

    # Environment information
    brain_name = env.brain_names[0]

    # Used for learning period to update the model weights
    global_step_number = 0

    for i_episode in range(1, n_episodes+1):
        
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations

        scores = np.zeros(num_agents)
        ma_ddpg.reset()
        
        while True:
            actions=ma_ddpg.act(states)
 
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            dones = env_info.local_done

            global_step_number += 1
            ma_ddpg.step(states, actions, rewards, next_states, dones, global_step_number)

            scores += rewards
            states = next_states

            if np.any(dones):
                break

        mean_score = np.mean(scores) # mean on 2 agents, required by the project
        scores_total.append(mean_score)
        scores_deque.append(mean_score) # mean on 2 agents for 100 episodes, results required
        #score_average_100epis = np.mean(scores_deque) # value = mean on 100 episodes 
        scores_global.append(np.mean(scores_deque)) # 
        
        s = (int)(time.time() - time_start)
        print('\rEpisode: {}, Score: {:.6f}, \tAverage Score: {:.6f}, Time: {:02}:{:02}:{:02} '\
                 .format(i_episode, mean_score,  np.mean(scores_deque), s//3600, s%3600//60, s%60), end="")
                
        if i_episode % print_every == 0:            
            print('\rEpisode: {}, Score: {:.6f}, \tAverage Score: {:.6f}, Time: {:02}:{:02}:{:02} '\
                 .format(i_episode, mean_score,  np.mean(scores_deque), s//3600, s%3600//60, s%60))

        if np.mean(scores_deque) >= 0.5 and not solved:
            print('\n*** Environment solved in {:d} episodes!\tAverage Score: {:.4f} ***\n'.format(
                i_episode, np.mean(scores_deque)))
            solved = True
            save_weights(dir_chkpoints)
            best_score = np.mean(scores_deque)

        if i_episode % save_every == 0 and solved:
            # Only save these weights if they are better than the ones previously saved
            if np.mean(scores_deque) > best_score:
                best_score = np.mean(scores_deque)
                save_weights(dir_chkpoints)
                s = (int)(time.time() - time_start) 
                print('*** Episode {}\tAverage Score: {:.4f}, Time: {:02}:{:02}:{:02} ***\n '\
                     .format(i_episode, np.mean(scores_deque), s//3600, s%3600//60, s%60))  
        

    return scores_total, scores_global

scores_total, scores_global = ma_ddpg_train()


Episode: 20, Score: 0.495000, 	Average Score: 0.290000, Time: 00:01:11 
Episode: 40, Score: -0.005000, 	Average Score: 0.268750, Time: 00:02:13 
Episode: 56, Score: 0.045000, 	Average Score: 0.246786, Time: 00:02:55

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
#import numpy as np
%matplotlib inline

fig = plt.figure()
ax = fig.add_subplot(111)

plt.plot(np.arange(1, len(scores_total) + 1), scores_total, label="Max Score")
plt.plot(np.arange(1, len(scores_global) + 1), 
    scores_global, label="Avg on 100 episodes") 
# This line indicates the score at which the environment is considered solved
plt.axhline(y=0.5, color="r",  label="Environment Solved") 

plt.legend(bbox_to_anchor=(1.05, 1)) 
plt.ylabel("Score")
plt.xlabel("Episode # ")
plt.show()

print('len of scores_total: ', len(scores_total), '\nlen of score_global: ',\
    len(scores_global))

fig.savefig('./Scores_tennis.png')

In [ ]:
def load(dir):    
        for i in range(num_agents):
            maddpg.agents[i].actor_local.load_state_dict(
                torch.load(os.path.join(dir, 'checkpoint_actor_{}.pth'.format(i))))
            maddpg.agents[i].critic_local.load_state_dict(
                torch.load(os.path.join(dir, 'checkpoint_critic_{}.pth'.format(i))))

            
def play(maddpg, env, num_games=11):
    """Tests the training results by having both agents play a match.
    Params
    ======
        maddpg (MADDPG): instance of MADDPG wrapper class
        env (UnityEnvironment): instance of Unity environment for testing
        num_games (int): number of games to be played
    """
    
    print("Agent #0: Red racket")
    print("Agent #1: Blue racket")
    print("---------------------")

    game_scores = [0 for _ in range(num_agents)]

    # Environment information
    brain_name = env.brain_names[0]

    for i_episode in range(1, num_games+1):
        env_info = env.reset(train_mode=False)[brain_name]   
        states = env_info.vector_observations
        scores = np.zeros(num_agents)

        t_step = 0
        
        while True:
            actions = maddpg.act(states)

            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            scores += rewards
            dones = env_info.local_done
            t_step += 1

            if np.any(dones):
                winner = np.argmax(scores)
                game_scores[winner] += 1
                if (t_step > 0):
                    t_step -= 1
                print('Game: {}, partial score: {},  Score #0: {:.2f}, Score #1: {:.2f}, Timesteps: {} '.
                       format(i_episode, game_scores, scores[0], scores[1], t_step))
                break

            states = next_states

    print("---------------------")
    print("Winner: Agent #{}".format(np.argmax(game_scores)))

In [ ]:
load(dir_chkpoints)
play(ma_ddpg, env, num_games=5)

In [ ]:
env.close()